In [1]:
%matplotlib inline
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *

# Loading your IBM Q account(s)
provider = IBMQ.load_account()

/opt/conda/lib/python3.8/site-packages/qiskit/providers/ibmq/ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


In [2]:
# python
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# qiskit
from qiskit import BasicAer
from qiskit.circuit.library import ZZFeatureMap
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.algorithms import QSVM
from qiskit.aqua.components.multiclass_extensions import AllPairs
from qiskit.aqua.utils.dataset_helper import get_feature_dimension

In [4]:
df = pd.read_csv("data2.csv")
df = df[['signs_of_mental_illness','flee','threat_other','age','poverty_rate','Median Income','Race_White']]

In [5]:
df.head()

,signs_of_mental_illness,flee,threat_other,age,poverty_rate,Median Income,Race_White
0,1,0,0,53.0,28.6,37072,0
1,0,0,0,47.0,14.9,65765,1
2,1,0,1,55.0,14.9,65765,1
3,0,0,1,23.0,17.3,45947,0
4,0,1,0,26.0,17.3,45947,1


In [29]:
df['Median Income'] = pd.to_numeric(df['Median Income'], errors='coerce')

In [31]:
# from sklearn import preprocessing

# normalize the data attributes
df_n = df.dropna()
df_n.iloc[:,3:] = (df_n.iloc[:,3:]-df_n.iloc[:,3:].mean()) / (df_n.iloc[:,3:].std())
df_n.head()

/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:1754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, val, pi)


,signs_of_mental_illness,flee,threat_other,age,poverty_rate,Median Income,Race_White
0,1,0,0,1.312747,1.121196,-0.727598,-0.940754
1,0,0,0,0.844691,-0.589722,1.070060,1.062476
2,1,0,1,1.468765,-0.589722,1.070060,1.062476
3,0,0,1,-1.027533,-0.289999,-0.171567,-0.940754
4,0,1,0,-0.793505,-0.289999,-0.171567,1.062476


In [32]:
df_n.describe()

,signs_of_mental_illness,flee,threat_other,age,poverty_rate,Median Income,Race_White
count,2123.000000,2123.000000,2123.000000,2.123000e+03,2.123000e+03,2.123000e+03,2.123000e+03
mean,0.246821,0.339143,0.308055,3.302953e-16,-4.792877e-15,2.399556e-16,-4.279823e-16
std,0.431263,0.473530,0.461798,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,0.000000,0.000000,0.000000,-2.353692e+00,-2.375570e+00,-2.293196e+00,-9.407540e-01
25%,0.000000,0.000000,0.000000,-7.935052e-01,-6.209429e-01,-6.224693e-01,-9.407540e-01
50%,0.000000,0.000000,0.000000,-1.694305e-01,-4.023010e-02,-1.203805e-01,-9.407540e-01
75%,0.000000,1.000000,1.000000,6.886721e-01,5.467270e-01,3.236305e-01,1.062476e+00
max,1.000000,1.000000,1.000000,3.887055e+00,7.090674e+00,7.975817e+00,1.062476e+00


In [33]:
class_labels = list(df_n.signs_of_mental_illness.unique())
class_labels

[1, 0]

In [137]:
train = pd.concat([df_n.loc[df.signs_of_mental_illness==0,].sample(100),df_n.loc[df.signs_of_mental_illness==1,].sample(100)])
test = pd.concat([df_n.loc[df.signs_of_mental_illness==0,].sample(100),df_n.loc[df.signs_of_mental_illness==1,].sample(100)])

In [42]:
train = df_n.sample(200)
test = df_n.sample(100)

In [43]:
sample_Total = train.iloc[:,1:].to_numpy() # numpy.ndarray
sample_Total[0:2,]

array([[ 0.        ,  0.        ,  2.5608961 , -2.11331251,  1.27668449,
         1.06247644],
       [ 0.        ,  0.        , -1.10554253, -0.26502217,  0.09501527,
        -0.94075401]])

In [44]:
d_train = {}
A = train.loc[train.signs_of_mental_illness==0,]
A = A.iloc[:,1:].to_numpy()
d_train[0] = A

B = train.loc[train.signs_of_mental_illness==1,]
B = B.iloc[:,1:].to_numpy()
d_train[1] = B

train_input = d_train # dict

In [45]:
d_test = {}
A = test.loc[test.signs_of_mental_illness==0,]
A = A.iloc[:,1:].to_numpy()
d_test[0] = A

B = test.loc[test.signs_of_mental_illness==1,]
B = B.iloc[:,1:].to_numpy()
d_test[1] = B

test_input = d_test # dict

In [46]:
temp = [test_input[k] for k in test_input]
total_array = np.concatenate(temp)

In [47]:
aqua_globals.random_seed = 100

backend = BasicAer.get_backend('qasm_simulator')
feature_map = ZZFeatureMap(feature_dimension=get_feature_dimension(train_input),
                           reps=2, entanglement='linear')
svm = QSVM(feature_map, train_input, test_input, total_array,
           multiclass_extension=AllPairs())
quantum_instance = QuantumInstance(backend, shots=1024,
                                   seed_simulator=aqua_globals.random_seed,
                                   seed_transpiler=aqua_globals.random_seed)

result = svm.run(quantum_instance)
for k,v in result.items():
    print(f'{k} : {v}')

Dataset has just two classes. Supplied multiclass extension will be ignored


testing_accuracy : 0.73
test_success_ratio : 0.73
predicted_labels : [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 1 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1
 0 0 1 1 1 1 0 0 0 0 1 0 0 0 1 0 1 1 0 1 0 0 1 0 1 1]
predicted_classes : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1]


In [41]:
train.loc[train.signs_of_mental_illness==0,].count()/train.shape[0]

signs_of_mental_illness    0.65
flee                       0.65
threat_other               0.65
age                        0.65
poverty_rate               0.65
Median Income              0.65
Race_White                 0.65
dtype: float64